<h1 style="text-align:center">Building an API for Your Model</h1>

#### What is an API?

An API allows you to interact with other computers. It takes a request and based on the information in that request it triggers the computer to run certain tasks and generate a specific output.  

<img src='images/api.png' width='400px'/>

<b>Example</b>

In [2]:
import requests

<img style='width:200px' src= 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Hotdog_-_Evan_Swigart.jpg/1200px-Hotdog_-_Evan_Swigart.jpg' />

In [3]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Hotdog_-_Evan_Swigart.jpg/1200px-Hotdog_-_Evan_Swigart.jpg'
# url = 'http://2.bp.blogspot.com/-kXJaeb6-Zuc/Uz2ZLpl_YeI/AAAAAAAAEFc/fFYzeDAh-SY/s1600/Peanut+Butter+and+Honey+Banana+Dog+002.JPG'
payload = {"image": url}
r = requests.post('http://ec2-3-16-164-62.us-east-2.compute.amazonaws.com/predict', data={'image':url}).json()

ConnectionError: HTTPConnectionPool(host='ec2-3-16-164-62.us-east-2.compute.amazonaws.com', port=80): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc534196f60>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
r

We are sending an image url to a computer (in this case, one of Amazon's). Our api then takes that image url and passes it to our trained model and returns a prediction. 

### How to build an API in Python

#### Flask

Goals
- Explain the difference between a POST and GET requests
- Create an API on our local machine 
- Send a request to a specific "route" and have it return a response

GET Methods - GET is the default method in each route. GET Methods recieve a request and return some information. When you type something into your browsers search bar you sending a GET request. 

<img style='width:300px' src='images/get_rm.gif' /> 

Flask is a web application framework built in Python. It allows us to set up a server that is listening for a request and when it recieves the request it returns an appropriate response. 

In [2]:
import flask

#initalize app
app = flask.Flask(__name__)

#when the route "/" recieves a request the function hello is run
@app.route("/predict")
def hello():
    return "Hello World!"

#run app
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2020 16:04:28] "GET / HTTP/1.1" 404 -


<img style='width:300px' src='images/get_rm.gif' /> 

#### POST Methods

In [3]:
import flask

#initalize app
app = flask.Flask(__name__)

#when the route "/" recieves a request the function hello is run
@app.route("/")
def hello():
    return "Hello World!"
#'/predict' reutrns 'Request recieved' when it recieves a POST request
@app.route("/predict", methods=["POST", "GET"])
def pred():
    if flask.request.method == "POST":
        years = flask.request.form['years']
        return f'Request recieved {years}'
    else:
        return 'GET'
#run app
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2020 16:04:36] "GET / HTTP/1.1" 200 -


#### Get a prediction from a model

In [5]:
import requests
response = requests.post('http://127.0.0.1:5000/predict')
response.content

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc577e040b8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [6]:
import pickle
import flask
import json
import numpy as np
#initalize app
app = flask.Flask(__name__)
#initialize model outside of route so it doesn't have to load everytime it recieves a request
model = pickle.load( open('model.pkl','rb'))

#when the route "/" recieves a request the function hello is run
@app.route("/")
def hello():
    return "Hello World!"


@app.route("/predict", methods=["POST"])
def pred():
    if flask.request.method == "POST":
        years = np.float(flask.request.form['years'])
        prediction = model.predict([[years]])
        data = {}
        data['predictions'] = prediction[0]
        return flask.jsonify(data)
#run app
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2020 16:04:48] "GET / HTTP/1.1" 200 -


In [7]:
import requests
response = requests.post('http://127.0.0.1:5000/predict', data={'years':1})
response.json()['predictions']

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc577e04cf8>: Failed to establish a new connection: [Errno 61] Connection refused',))

<b>Resources</b>

https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html <br/>
https://www.pyimagesearch.com/2018/01/29/scalable-keras-deep-learning-rest-api/